In [2]:
from dotenv import load_dotenv
from langchain.llms import OpenAI

# load environment variables from .env file
load_dotenv()

In [29]:
class ControllerModel():
    def __init__(self):
        self.pre_prompt = """We are in the situation of a case interview for applying to a job in consulting. You are taking the role of the interviewer in this case. You are there to practice solving case studies with the interviewee. You are provided with a reference solution to the case. You are not tasked to solve the case yourself but rather guide the candidate throughout the case. Important things for you to remember
            \n - You are supposed to help the candidate but not provide with with the solutions. The candidate must be able to solve the case on its own.
            \n - The candidate must not match the reference solution one to one but should provide most information.
            \n - after each step wait for the candidate to answer the question. You are never to take the role of the candidate and answer questions yourself!
            \n - Never automatically add something to the responses of the candidate. Only react towards what the candidate is writing.
            \n - You are provided with the full history of the conversation after the # Case Interview tag. Tags are used to show who said what in the conversation. Possible tags are 1) Candidate: 2) Interviewer: 3) Command: 4) State:
            \n - You are only supposed to use the Interviewer or System tag. Use the Interviewer tag whenever you are talking to the Candidate. Ocasionally a Command will be used to ask you something about the state of the interview (example: Command: Which section of the interview are we currenlty in?). Use the State tag to respond to these commands.
            \n - Command: and State: tags are not shown to the candidate
            \n - The command tag is used to provide additional commands to you. Pay attention to these commands when continuing the conversation. 
            """
        
        self.task_specific_prompt = """
            A case study consits of different parts. There are experts for different parts. Your task is to control the interaction with the candidate. This includes:
            \n - Identifying which part of the case interview we are currently in
            \n - Delegating the responsibility for each part to the corresponding expert
            \n - Making Transitions between the different parts of the case interview. This means, you ask the candidate where he wants to continue. For this you are provided with a reference structure of the case. If the next step of the candidate
                overlap with some part of the reference solution, let the candidate continue there. Otherwise, tell the candidate that we instead want to focus on something else and guide him to one section of the reference solution.
        """

        self.case_information = """# Reference Information about the case
            \n ## Problem Statement: A leading biotech company is developing a treatment for Alzheimer's disease. This ground-breaking treatment is different from any other Alzheimer's treatment on the market because it actually slows the progression of the disease, rather than just treating its symptoms. The company’s executive team is concerned about the results of a high-level internal feasibility study that flagged a potential risk to the launch of this treatment – a rumored shortage of infusion capacity in the US. Given that the Alzheimer's treatment is designed to be administered via infusion, such a shortage would severely hamper the market acceptance and hence the financial rewards from the treatment. In preparation for the launch of this treatment, the company has hired you to help them figure out the extent of the expected shortfall, and how they should respond.
            \n ## Additional Information:  [
                "Infusion refers to inserting the medicine directly into a patients bloodstream via IV (intravenous) application, ideally through the patient’s arm.",
                "The treatment will be launched in the US alone.",
                "The client has not yet estimated how big the infusion shortfall will be.",
                "The client does not have any strategies to mitigate the shortfall.",
                "Most other Alzheimers medications are delivered as oral pills.",
                "The treatment (if approved by the FDA) would come to market in about 2 years."
            ]"""

        self.reference_structure = """
            # Reference Structure of the Case
            The interview consits of multiple stages that must be completed sequentially. The stages are:
            \n1. Introduction: Provide the problem statement to the candidate. The candidate can ask clarifying questions to better understand the task.
            \n2. Framework: The candidate should come up with a framework to solve the case. The framework should be structured and cover all important aspects of the case.
            \n3. Questions: The candidate should work through the questions in the case. The order is not important but the candidate should work through all questions. For this case the questions are:
            \n  3.1. Question 1 - Numeracy: How much do you estimate the shortfall of infusion capacity will be in the US for this treatment when it enters the market?
            \n  3.2 Question 2 - Brainstorming: The client intends to introduce the new treatment to the market in 2 years. How can the biotech firm mitigate the infusion capacity shortfall and what immediate actions can they take?
        """

        self.conversation_history = [self.pre_prompt, self.task_specific_prompt, self.case_information, self.reference_structure]
        self.llm = OpenAI()
    def run_llm(self):
        return self.llm("\n\n".join(self.conversation_history))
    
    def open_case(self):
        opener = "Hello, I am the interviewer for this case if you are ready, I would provide you with the problem statement right now. Problem Statement: A leading biotech company is developing a treatment for Alzheimer's disease. This ground-breaking treatment is different from any other Alzheimer's treatment on the market because it actually slows the progression of the disease, rather than just treating its symptoms. The company’s executive team is concerned about the results of a high-level internal feasibility study that flagged a potential risk to the launch of this treatment – a rumored shortage of infusion capacity in the US. Given that the Alzheimer's treatment is designed to be administered via infusion, such a shortage would severely hamper the market acceptance and hence the financial rewards from the treatment. In preparation for the launch of this treatment, the company has hired you to help them figure out the extent of the expected shortfall, and how they should respond.\nPlease feel free to ask any questions that you have about the problem statement."
        self.conversation_history.append(f"Interviewer: {opener}")
        print(opener)

    def add_interaction(self, interaction):
        self.conversation_history.append(f"Candidate: {interaction}")
        
        self.conversation_history.append("Command: Command: At which section of the interview are we currently in. Provide the result in the exact format Section Number: <section number> Section Name: <section name>.")
        result = self.run_llm()
        print(f"Current Section: {result}")

        


    
    


In [30]:
controller_model = ControllerModel()
controller_model.open_case()

controller_model.add_interaction("I don't think I have any more questions. Would it be ok If I take some minutes to come up with a framework?")


Hello, I am the interviewer for this case if you are ready, I would provide you with the problem statement right now. Problem Statement: A leading biotech company is developing a treatment for Alzheimer's disease. This ground-breaking treatment is different from any other Alzheimer's treatment on the market because it actually slows the progression of the disease, rather than just treating its symptoms. The company’s executive team is concerned about the results of a high-level internal feasibility study that flagged a potential risk to the launch of this treatment – a rumored shortage of infusion capacity in the US. Given that the Alzheimer's treatment is designed to be administered via infusion, such a shortage would severely hamper the market acceptance and hence the financial rewards from the treatment. In preparation for the launch of this treatment, the company has hired you to help them figure out the extent of the expected shortfall, and how they should respond.
Please feel fre